In [2]:
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np
import os

In [74]:
urls=["https://search.shopping.naver.com/catalog/24452740523?query=%EC%95%A0%ED%94%8C&NaPm=ct%3Dksi7vtzc%7Cci%3D401841aa5a5985115a341d57cb55ec0f297a5b3f%7Ctr%3Dslsl%7Csn%3D95694%7Chk%3Dec5661bbefd2ead640763fa358be70960b126223"]

In [75]:
for url in urls:
    
    shoppingmall_review = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[1]/ul/li[4]/a"

    shppingmall_review2 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[2]/div/div[2]/ul/li[4]/a"
#                                                                v                                ^
    point5 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[2]/a"
    point4 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[3]/a"
    point3 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[4]/a"
    point2 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[5]/a"
    point1 = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[6]/a"
    points_path = [point4, point3, point2, point1]

    title_path = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[2]/div[1]/em"
    text_path = "/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/ul/li[1]/div[2]/div[1]/p"

    header = {'User-Agent': ''}
    d = webdriver.Chrome('chromedriver.exe') # webdriver = chrome
    d.implicitly_wait(3)
    d.get(url)
    req = requests.get(url)
    html = req.text
    soup = BeautifulSoup(html, "html.parser")

    a = d.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div[1]/h2")
    a = a.text #상품 제목


    elem = d.find_element_by_tag_name("body")
    elem.send_keys(Keys.PAGE_DOWN)
    sleep(2)

    d.find_element_by_xpath(shoppingmall_review).click()
    sleep(2)

    elem.send_keys(Keys.PAGE_DOWN)
    sleep(2)

    full_list = [[],[]]


    for i in range(len(points_path)):
        point = []
    
        d.find_element_by_xpath(shppingmall_review2).click()
        sleep(2)

    
        d.find_element_by_xpath(points_path[i]).click()
        print(points_path[i])
        sleep(2)
    
        page = 1
        starpoint = 4-i
    
        while True:
            html = d.page_source
            dom = BeautifulSoup(html, "lxml")
            titles_raw = dom.find_all("em", {"class":"reviewItems_title__39Z8H"})
            texts_raw = dom.find_all("p", {"class":"reviewItems_text__XIsTc"})


            try:
                d.find_element_by_xpath("/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[3]/a["+str(page)+"]").click()

                sleep(1)
                if page == 11:
                    page = 2


                html = d.page_source
                dom = BeautifulSoup(html, "lxml")
                titles_raw = dom.find_all("em", {"class":"reviewItems_title__39Z8H"})
                texts_raw = dom.find_all("p", {"class":"reviewItems_text__XIsTc"})
            
            
                for text_raw in texts_raw:
                    full_list[0].append(text_raw.text)
                    full_list[1].append(starpoint)
                    point.append(starpoint)

                if len(texts_raw) != 20:
                    break
        
                page = page + 1
            
                if len(point) > 400:
                    break
            
                sleep(4)
            except:
                for text_raw in texts_raw:
                    full_list[0].append(text_raw.text)
                    full_list[1].append(starpoint)
                    point.append(starpoint)
                break



    d.quit()
            
    df = pd.DataFrame(full_list)
    df = df.T
    df.columns = ["review", "star"]
    df.to_csv("C:/690009/TP2/final_project/member/NTJ/crawlerNSstar/data/"+a+".csv", encoding='utf-8-sig')

/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[3]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[4]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[5]/a
/html/body/div/div/div[2]/div[2]/div[2]/div[3]/div[6]/div[2]/div[2]/ul/li[6]/a


In [76]:
df

,review,star
0,색은 정말 .. 이번이 레전드는 맞아요 사진보다 옅고 영롱하고 예쁘고 취저하고 혼자...,4
1,"생각한 것보다 좀 더 색이 예쁘네요. 사진에서 색이 잘 안담기는데, 실물 색이 좀 ...",4
2,"화이트가 없어서 망설였는데, 갑자기 화이트 물량이 풀려서 바로 샀습니다! 원래 사용...",4
3,핸드폰은 아주 만족합니다.카메라 렌즈 프로텍터랑 케이스랑은 전혀 맞지않아서 렌즈 프...,4
4,공홈에 잠시 품절이라 프리스비에서 샀는데배송시작이라고 안떠서 취소할랬더니엄청 빠르게...,4
...,...,...
94,배송은 느리고 문의 답변 속도도 느리고.. 저렴하게 샀지만 너무 답답했습니다. 상품...,2
95,상품은 좋아요그치만 포장은 조금 아쉬워요.생각보다 너무 큰 박스에 뾱뾱이 비닐커버 ...,2
96,프리습에서 애플 처음 구매했는데 애플공식 대리점치고 사후처리는 비공식같은 곳이네요,2
97,비쌈 이런거 사주니깐 애플이 동양사람들 소비자들 우습게 봄,1
